In [1]:
import pandas as pd
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import joblib
import math

In [2]:
df = pd.read_csv('data/data_abono/labeled.csv')
print(len(df))
# train = df[:math.floor(0.7*len(df))]
# test  = df[math.floor(0.7*len(df)):]
# print(len(df), len(train), len(test))
# df = []

228


In [3]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq):
    x = []
    y = []
    for i in range(len(arq)):
        x.append(extract_features(arq['text'][i].split()))
        y.append(arq['labels'][i].split())
    return x, y

x, y = separate_cols(df)
train_x = x[:math.floor(0.8*len(df))]
train_y = y[:math.floor(0.8*len(df))]
test_x = x[math.floor(0.8*len(df)):]
test_y = y[math.floor(0.8*len(df)):]

len(df), len(train_x), len(train_y), len(test_x), len(test_y)

(228, 182, 182, 46, 46)

In [4]:
model = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd', 
    c2=1,
    max_iterations=100, 
    all_possible_transitions=True,
    verbose=False
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

In [16]:
labels = list(model.classes_)
labels.remove('O')

f1 = metrics.flat_f1_score(test_y, y_pred, 
                      average='weighted', labels=labels)
print(f1)

0.9702607256014307


In [17]:
print(metrics.flat_classification_report(
    test_y, y_pred, labels=labels, digits=3
))

                                 precision    recall  f1-score   support

                         B-Nome      0.985     0.977     0.981       133
                         I-Nome      0.983     0.988     0.985       408
                B-cargo_efetivo      0.981     0.962     0.971        53
                I-cargo_efetivo      0.987     0.983     0.985       238
                    B-matricula      1.000     0.985     0.992       133
                    I-matricula      1.000     1.000     1.000        98
                        B-siape      1.000     1.000     1.000        11
                        I-siape      1.000     1.000     1.000        32
                     B-vigencia      1.000     0.962     0.981       133
                     I-vigencia      1.000     0.966     0.983       409
             B-processo_gdf_sei      0.931     1.000     0.964       135
             I-processo_gdf_sei      1.000     1.000     1.000        84
    B-fundamento_legal_do_abono      0.970     0.9

E:\Instalations\envs\ner\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass labels=['B-Nome', 'I-Nome', 'B-cargo_efetivo', 'I-cargo_efetivo', 'B-matricula', 'I-matricula', 'B-siape', 'I-siape', 'B-vigencia', 'I-vigencia', 'B-processo_gdf_sei', 'I-processo_gdf_sei', 'B-fundamento_legal_do_abono', 'I-fundamento_legal_do_abono', 'B-padrao', 'I-padrao', 'B-quadro_permanente-suplementar', 'I-quadro_permanente-suplementar'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [19]:
# Save trained model
joblib.dump(model, "abono_ner.pkl")

['abono_ner.pkl']